In [8]:
import requests
import pandas as pd
import time

def get_basic_income_amounts(policy_id):
    # API endpoint URL
    api_url = f"https://api.policyengine.org/us/policy/{policy_id}"

    # Making a GET request to the API
    response = requests.get(api_url)

    if response.status_code == 200:
        # Parsing the JSON response
        policy_data = response.json()

        # Extracting and renaming the basic income amounts
        basic_income_amounts = {}
        age_group_mapping = {
            "gov.contrib.ubi_center.basic_income.amount.person.by_age[0].amount": "ubi_0_5",
            "gov.contrib.ubi_center.basic_income.amount.person.by_age[1].amount": "ubi_6_17",
            "gov.contrib.ubi_center.basic_income.amount.person.by_age[2].amount": "ubi_18_64",
            "gov.contrib.ubi_center.basic_income.amount.person.by_age[4].amount": "ubi_65_plus",
            "gov.contrib.ubi_center.basic_income.amount.person.disability": "ubi_disability"
        }

        for key, value in policy_data["result"]["policy_json"].items():
            if key in age_group_mapping:
                # Assuming the amount is the first value in the dictionary
                amount = next(iter(value.values()))
                basic_income_amounts[age_group_mapping[key]] = amount
        return basic_income_amounts
    else:
        return None

# Read the CSV file into a DataFrame
df = pd.read_csv('loss_by_rate.csv')

def get_json_from_id(id):
    res = requests.get(f"https://api.policyengine.org/us/economy/{id}/over/2?time_period=2024&region=enhanced_us").json()

    while res["status"] == "computing":
        time.sleep(5)
        res = requests.get(f"https://api.policyengine.org/us/economy/{id}/over/2?time_period=2024&region=enhanced_us").json()
    
    return res["result"]


In [10]:
# Fetch and add basic income amounts for each reform ID
# json_data = {}

for i, row in df.iterrows():
    reform_id = int(row['rounded_ubi_api_id'])
    print(reform_id)
    amounts = get_basic_income_amounts(reform_id)
    if amounts:
        for column_name, amount in amounts.items():
            df.loc[i, column_name] = amount
    # json_data[reform_id] = get_json_from_id(reform_id)


df.to_csv('ubi_amounts.csv', index=False)
# print(json_data)

42843
42844
42845
42846
42847
42848
42849
42850
42851
42852
42853
42854
42855
42856
42857
42858
42859
42860
42861
42862
42863
42864
    flat_tax_rate      loss  api_id  rounded_ubi_api_id  ubi_0_5  ubi_6_17  \
0            0.34  0.061239   42757               42843   3720.0    3480.0   
1            0.33  0.055422   42758               42844   3960.0    3360.0   
2            0.35  0.093698   42759               42845   5280.0    4560.0   
3            0.32  0.053071   42760               42846   4680.0    3240.0   
4            0.36  0.179060   42762               42847   4680.0    4680.0   
5            0.31  0.051001   42764               42848   3720.0    3240.0   
6            0.37  0.594883   42766               42849  10920.0    8280.0   
7            0.30  0.049753   42767               42850   4080.0    3000.0   
8            0.38  1.760803   42768               42851   9720.0   11160.0   
9            0.20  0.122087   42769               42852   3360.0    3000.0   
10        

In [24]:
import json

# Write the dictionary to disk as JSON
with open("json_data.json", "w") as file:
    json.dump(json_data, file)


In [11]:
import json

# Load the JSON file as a dictionary
with open("json_data.json", "r") as file:
    json_data = json.load(file)

# Print the dictionary
print(json_data)


{'42843': {'budget': {'baseline_net_income': 11960959952723.936, 'benefit_spending_impact': 1647098001884.5273, 'budgetary_impact': 12017397521.239746, 'households': 134663334.54167563, 'state_tax_revenue_impact': 0.0, 'tax_revenue_impact': 1659115399405.767}, 'decile': {'average': {'1': 4813.01719387118, '10': 1428.3033911147122, '2': 3261.318981102547, '3': 1950.5396154471614, '4': 818.4855029022159, '5': -865.1654260552383, '6': -2935.898054463781, '7': -5877.8195034968585, '8': -7798.810336516801, '9': -10840.074278544571}, 'relative': {'1': 1.5998312288534762, '10': 0.003080027101791351, '2': 0.1012305314545103, '3': 0.044381176720653115, '4': 0.015079141975563282, '5': -0.013332626678707271, '6': -0.0380626382755641, '7': -0.06336100026324, '8': -0.06844421501656137, '9': -0.0735253185774422}}, 'detailed_budget': {}, 'inequality': {'gini': {'baseline': 0.5274872779314961, 'reform': 0.5032475234003438}, 'top_10_pct_share': {'baseline': 0.38444705444755384, 'reform': 0.381699102135

In [12]:
import plotly.express as px

# Melt the DataFrame to make it suitable for Plotly
melted_df = df.sort_values("flat_tax_rate").melt(id_vars=['flat_tax_rate'], 
                    value_vars=['ubi_0_5', 'ubi_6_17', 'ubi_18_64', 'ubi_65_plus', 'ubi_disability'],
                    var_name='UBI Category', value_name='Amount')

# Create the plot
fig = px.line(melted_df, x='flat_tax_rate', y='Amount', color='UBI Category',
              labels={'flat_tax_rate': 'Flat Tax Rate', 'Amount': 'UBI Amount'},
              title='UBI Amounts by Flat Tax Rate')

# Show the plot
fig.show()

In [15]:
import pandas as pd

def create_dataframes_from_json(json_data, df):
    dataframes = {}
    for api_id, metrics in json_data.items():
        for metric_name, metric_data in metrics.items():
            if metric_name not in dataframes:
                dataframes[metric_name] = pd.DataFrame()

            # Flatten the metric data and add API ID
            flat_data = pd.DataFrame([{**{"rounded_ubi_api_id": api_id}, **metric_data}])
            dataframes[metric_name] = pd.concat([dataframes[metric_name], flat_data], ignore_index=True)

    # Ensure data type consistency for merging
    df['rounded_ubi_api_id'] = df['rounded_ubi_api_id'].astype(str)

    # Merge each dataframe with the df dataframe to add the flat_tax_rate
    for metric_name, metric_df in dataframes.items():
        metric_df['rounded_ubi_api_id'] = metric_df['rounded_ubi_api_id'].astype(str)
        merged_df = metric_df.merge(df[['rounded_ubi_api_id', 'flat_tax_rate']], on='rounded_ubi_api_id', how='left')
        # Sort the DataFrame by flat_tax_rate
        dataframes[metric_name] = merged_df.sort_values(by='flat_tax_rate')

    return dataframes

# Assuming 'json_data' is your JSON data and 'df' is your existing DataFrame
dfs = create_dataframes_from_json(json_data, df)

# Accessing and printing a specific dataframe, for example, 'budget'
budget_df = dfs.get('budget', None)
if budget_df is not None:
    print(budget_df)


   rounded_ubi_api_id  baseline_net_income  benefit_spending_impact  \
9               42852         1.196096e+13             6.494602e+10   
10              42853         1.196096e+13             6.913572e+10   
11              42854         1.196096e+13             9.877809e+10   
12              42855         1.196096e+13             1.924121e+11   
13              42856         1.196096e+13             3.300214e+11   
7               42850         1.196096e+13             1.132128e+12   
5               42848         1.196096e+13             1.266689e+12   
3               42846         1.196096e+13             1.395907e+12   
1               42844         1.196096e+13             1.525258e+12   
0               42843         1.196096e+13             1.647098e+12   
2               42845         1.196096e+13             1.797118e+12   
4               42847         1.196096e+13             1.911301e+12   
6               42849         1.196096e+13             2.042417e+12   
8     

In [16]:
px.line(budget_df, "flat_tax_rate", "tax_revenue_impact")

In [17]:
import pandas as pd
import plotly.express as px

def calculate_percentage_change(dataframe, column_name):
    data = dataframe[column_name].apply(pd.Series)
    data['% change'] = ((data['reform'] - data['baseline']) / data['baseline'])
    data['flat_tax_rate'] = dataframe['flat_tax_rate']
    return data[['flat_tax_rate', '% change']].rename(columns={'% change': f'% change {column_name}'})

# Assuming dfs['inequality'] is your DataFrame
gini_change = calculate_percentage_change(dfs['inequality'], 'gini')
top_10_pct_share_change = calculate_percentage_change(dfs['inequality'], 'top_10_pct_share')
top_1_pct_share_change = calculate_percentage_change(dfs['inequality'], 'top_1_pct_share')

# Combine the data
combined_data = gini_change
combined_data = combined_data.merge(top_10_pct_share_change, on='flat_tax_rate')
combined_data = combined_data.merge(top_1_pct_share_change, on='flat_tax_rate')

fig = px.line(combined_data, x='flat_tax_rate', y=['% change gini', '% change top_10_pct_share', '% change top_1_pct_share'],
              labels={'value': '% Change', 'variable': 'Metric'},
              title='Percentage Change in Inequality Metrics with Respect to Tax Rate')
fig.show()

In [18]:
def calculate_percentage_change_poverty(dataframe, column_name):
    data = dataframe[column_name].apply(pd.Series)
    for category in ['adult', 'all', 'child', 'senior']:
        data[f'% change {category}'] = ((data[category].apply(lambda x: x['reform']) - 
                                         data[category].apply(lambda x: x['baseline'])) / 
                                         data[category].apply(lambda x: x['baseline'])) * 100
    data['flat_tax_rate'] = dataframe['flat_tax_rate']
    return data[['flat_tax_rate', f'% change adult', f'% change all', f'% change child', f'% change senior']]

# Assuming dfs['poverty'] is your DataFrame
poverty_change = calculate_percentage_change_poverty(dfs['poverty'], 'poverty')


fig = px.line(poverty_change, x='flat_tax_rate', y=[f'% change {category}' for category in ['adult', 'all', 'child', 'senior']],
              labels={'value': '% Change', 'variable': 'Category'},
              title='Percentage Change in Poverty Metrics with Respect to Tax Rate')
fig.show()


In [19]:
deep_poverty_change = calculate_percentage_change_poverty(dfs['poverty'], 'deep_poverty')


fig = px.line(deep_poverty_change, x='flat_tax_rate', y=[f'% change {category}' for category in ['adult', 'all', 'child', 'senior']],
              labels={'value': '% Change', 'variable': 'Category'},
              title='Percentage Change in Deep Poverty Metrics with Respect to Tax Rate')
fig.show()


In [28]:
import pandas as pd
import plotly.express as px

# Define a function to determine increase (1) or decrease (-1)
def increase_or_decrease(value):
    return 1 if value > 0 else -1

# Combine the percentage change data from poverty, deep poverty, and inequality
# Assuming dfs['poverty'] and other DataFrames are already defined
combined_change = dfs['poverty'][['flat_tax_rate']].copy()

# Assume you have other dataframes like gini_change, top_10_pct_share_change, etc.
for dataframe in [gini_change, top_10_pct_share_change, top_1_pct_share_change, poverty_change, deep_poverty_change]:
    for col in dataframe.columns:
        if col != 'flat_tax_rate':
            combined_change[col] = dataframe[col].apply(increase_or_decrease)

import pandas as pd
import plotly.figure_factory as ff

# Assume combined_change is a DataFrame with the given structure
# Mapping the percentage change to colors
combined_change_mapped = combined_change.copy()
for col in combined_change.columns:
    if col != 'flat_tax_rate':
        combined_change_mapped[col] = combined_change[col].map({1: 'Increase', -1: 'Decrease'})

import pandas as pd
import plotly.figure_factory as ff

# Rest of your code where you define combined_change and combined_change_mapped...

# Preparing the data for the heatmap
heatmap_data = combined_change_mapped.set_index('flat_tax_rate').T

# The x and y coordinates for the heatmap should be lists or arrays
x_values = combined_change_mapped['flat_tax_rate'].tolist()
y_values = heatmap_data.index.tolist()

# The z values and annotations must be 2D arrays that match the length of x and y
z_values = heatmap_data.values.T  # Transposing to match the dimensions
annotations = z_values  # This should be the same shape as z_values

# Creating the heatmap using Figure Factory to properly display categorical data
fig = ff.create_annotated_heatmap(
    z=z_values,
    x=x_values,
    y=y_values,
    annotation_text=annotations,
    colorscale=['blue', 'gray'],
    showscale=False
)

# Update the layout to remove the color axis scale (legend)
fig.update_layout(
    title='Increase or Decrease in Metrics with Respect to Tax Rate',
    xaxis=dict(title='Flat Tax Rate', side='top', tickmode='array', tickvals=x_values, ticktext=[f"{x:.2f}" for x in x_values]),
    yaxis=dict(title='Metric', tickmode='array', tickvals=list(range(len(y_values))), ticktext=y_values),
    margin=dict(l=150, t=50)  # Adjust margins to fit y-axis labels
)

# Show the figure
fig.show()


PlotlyError: oops, the x list that you provided does not match the width of your z matrix 

In [30]:
combined_change.to_csv('combined_change.csv', index=False)